In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
from keras.models import load_model
import pickle
import random
import dlib
import time
import shutil
from collections import Counter

#get_ipython().magic('matplotlib inline')
#get_ipython().magic('load_ext autoreload')
#get_ipython().magic('autoreload 2')
#%reload_ext autoreload
#np.set_printoptions(threshold=np.nan)


Using TensorFlow backend.


In [2]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula 
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square((tf.subtract(anchor,positive))),axis=-1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
    basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)
    loss = tf.reduce_sum(tf.maximum(basic_loss,0))
    
    return loss


In [3]:
# FaceNet model used to get the 128 encodings of the input image.
# FaceNet model is loaded mentioning its loss function and saved it with name-> model
model = load_model('model.h5', custom_objects={'triplet_loss': triplet_loss})

In [4]:
# Face_Recognition Database is loaded and named it as data_base
with open('C:\\Users\\Owner\\Desktop\\p2/data_base.pickle', 'rb') as handle:
    data_base = pickle.load(handle)

In [5]:
def page_1(id_):    # First page which opens after login
    data=pd.read_csv("C:/Users/Owner/Desktop/p2/user_data.csv")
    print(data.loc[int(id_-1),'name'])
    print("select an option")
    print("1.To know your basic details")
    print("2.to add a prescription")
    print("3.To view prescriptions")
    print("4.to log out")
    e=int(input())
    if e==1:
        print("Your account details:")
        print(data[data['ID']==float(id_)])
    elif e==2:
               #print("Enter the data to save it")
               #pres=str(input())
               #with open()
               pass
    elif e==4:
        print("Sucessfully logged out")
        exit()
               
def who_is_it(encoding, data_base, model): # who_is_it() function returns the ID of a person
   # Initialize "min_dist" to a large value, say 100 
    min_dist = 100
    id_=0
    for i in range(0,len(data_base)):
        
        j,database=data_base[i]
        for (name, db_enc) in database[0].items():
            # Computed L2 distance between the target "encoding" and the current "emb" from the database.
            dist = np.linalg.norm(db_enc-encoding)
            #print(name,":",dist)
            # If this distance is less than the min_dist, then  min_dist is assigned to dist, and identity to name. 
            if dist < min_dist:
                min_dist = dist
                identity = name
                id_=j
           
        
    #print(min_dist)
    if min_dist > 0.7:
        print("Not in the database.")
        print("Please try again")
        main(data_base)
    else:
        pass
    return id_
               
        
  #  return min_dist, identity
               
               
def relight(img,light=1,bias=0):   # Function relight enlights or delights the face randomly 
    w=img.shape[1] 
    h=img.shape[0]
    for i in range(0,w):
        for j in range(0,h):
            for c in range(3):
                tmp=int(img[j,i,c]*light+bias)
                if tmp>255:
                    tmp=255
                elif tmp<0:
                    tmp=0
                img[j,i,c]=tmp
    return img
# '''               
# def verify(image_path, data_base, model,in_dir,data,t):
#     encoding = img_to_encoding(image_path,model)
#     min_dist = 0.7
#     for i in list(range(0,len(data_base))):
#         j,database=data_base[i]
#         for (name, db_enc) in database[0].items():
#             dist = np.linalg.norm(db_enc-encoding)
#             if dist < min_dist:
#                 min_dist = dist
#                 identity = name
#                 id_=j
#                 break

#     if min_dist >= 1.0:
#         print("Sucessfully Registered")
#         data.to_csv("C:/Users/Owner/Desktop/p2/user_data.csv",index=False)
#     else:
#         shutil.rmtree(in_dir)
#         data.drop(t,inplace=True)
        
#         print("You are already registered User.")
#         main(data_base)
# '''

In [6]:
def register(data_base):
    #data={'ID':[None],'name':[None],'email_id':[None],'Gender':[None],'Phone':[None]}
    #data=pd.DataFrame(data)
    data=pd.read_csv("C:/Users/Owner/Desktop/p2/user_data.csv")
    t=data['ID'].shape[0]
    o_dir='C:/Users/Owner/Desktop/p2/face_data'
    img_size=96
    t=(pd.DataFrame(data['ID'])).shape[0]
    data.loc[t,'ID']=int(t+1)
    print("Enter the name:")
    data.loc[t,'name']=str(input())
    print("Enter your email_id:")
    data.loc[t,'email_id']=str(input())
    print("Enter your Gender (M/F):")
    data.loc[t,'Gender']=str(input())
    print("Enter your phone number:")
    data.loc[t,'Phone']=int(input())
    detector=dlib.get_frontal_face_detector()
    print("Look at the camera to give face data ")
    print("Don't move untill a message is poped")
    time.sleep(1)
    in_dir=os.path.join(o_dir,data.loc[t,'name']+'_'+str(int(data.loc[t,'ID'])))  #Creating a directory for user ... name followed by his ID
    if not os.path.exists(in_dir):
        os.makedirs(in_dir)
    camera=cv2.VideoCapture(0)
    index=1
    while(index<=40) :
        success,image=camera.read()
        d=detector(image,1)
        for i,d in enumerate(d):
            x1=d.top() if d.top() >0 else 0
            y1=d.bottom() if d.bottom()>0 else 0
            x2=d.left() if d.left()>0 else 0
            y2=d.right() if d.right()>0 else 0
            
            face=image[x1:y1,x2:y2]
            face=relight(face,random.uniform(0.5,1.5),random.randint(-50,50))
            face=cv2.resize(face,(img_size,img_size))
            cv2.imwrite(in_dir+'/'+str(data.loc[t,'name'])+str(index)+'.jpg',face)
            index=index+1
    camera.release()
    print('Face data is captured!!!!!!!')
    print("Please wait ")
    #verify(in_dir+'/'+str(data.loc[t,'name'])+str(index-1)+'.jpg',data_base,model,in_dir,data,t)
    data.to_csv('user_data.csv',index=False)
    database={}
    k=1
    for p in os.listdir(in_dir):
        database[data.loc[t,'name']+'_'+str(k)]=img_to_encoding(os.path.join(in_dir,p),model)
        k=k+1
    z=np.zeros(128)
    for j,i in database.items():
        z=z+np.abs(i)
    z=z/40
    z={data.loc[t,'name']:z}
    u=[int(data.loc[t,'ID']),[z]]
    data_base.append(u)
    with open('data_base.pickle', 'wb') as handle:
        pickle.dump(data_base, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('data_base.pickle', 'rb') as handle:
        data_base = pickle.load(handle)

    
    main(data_base)
    

In [7]:
# Model to detect the liveness of the person is liveness_dector.h5, which is loaded and named it as mod
# Model's output labels are 0 and 1 (0->fake image and 1-> real image)
mod=load_model('C:\\Users\\Owner\\Desktop\\p2/liveness_detector.h5')

In [18]:
def login(data_base):   
    e=0
    while e<1:
        detector=dlib.get_frontal_face_detector()  # detects the face in a picture
        img_size=96
        camera=cv2.VideoCapture(0)   # Captures the picture
        index=1
        while(index<=40):  # takes 15 images 
            success,image=camera.read()
            d=detector(image,1)
            for i,d in enumerate(d):
                x1=d.top() if d.top() >0 else 0
                y1=d.bottom() if d.bottom()>0 else 0
                x2=d.left() if d.left()>0 else 0
                y2=d.right() if d.right()>0 else 0

                face=image[x1:y1,x2:y2]
                face=relight(face,random.uniform(0.5,1.5),random.randint(-50,50))
                face=cv2.resize(face,(img_size,img_size))
                cv2.imwrite('C:\\Users\\Owner\\Desktop\\p2/login_images'+'/'+'li'+str(index)+'.jpg',face)
                index=index+1
        # Liveness detection is done using the below code  
        img=[]
        x_dir='C:\\Users\\Owner\\Desktop\\p2\\login_images'
        for i in os.listdir(x_dir):
            x=np.array(cv2.imread(os.path.join(x_dir,i)))/255
            x=np.expand_dims(x,axis=0)
            img.append(x)   
        pred=[]
        for i in range(0,index-1):
            x=mod.predict(img[i])
            pred.append(x)
        p=[]
        for i in range(0,index-1):
            p.append(np.argmax(pred[i]))
        k=[i for i,j in Counter(p).items() if j==max(Counter(p).values())]

        if k[0]==1:
            print("Real image")
        else:
            print("Fake image")
            e=e+1
            continue # If it detects as a fake image , it will exit and asks user to try again              
        camera.release()  
        dir_='C:\\Users\\Owner\\Desktop\\p2/login_images'  
        k=1
        pk=[]
        dat={}
        #Below shown the code to get the Id of a person
        for p in os.listdir(dir_):
            dat[str(p)+'_'+str(k)]=img_to_encoding(os.path.join(dir_,p),model)
            k=k+1
        # Loop to divide 40 images into 5 sets , each consisting of 8 images
        for x,y in [(0,8),(8,16),(16,24),(24,32),(32,40)]:
            z=np.zeros(128)
            for j,i in list(dat.items())[x:y]:
                z=z+np.abs(i)
            z=z/8  # Average of 8 images in each set is computed and passed as a augument to who_is_it() function.
            pk.append(who_is_it(z,data_base,model)) # Set of id's are appended to the list named-> pk
        print('id :',pk)
        k=[k for k,v in Counter(pk).items() if v==max(Counter(pk).values())] # To obtain the id with maximum count
        # k is the list containing id
        id_=k[0]
        print("Welcome")
        page_1(id_) # page function is called and passing the Id as a augument
        e=e+1

In [19]:
def main(data_base):
    print("Select one option")
    print("1.Already registered")
    print("2.New User")
    print("3.Exit")
    num=int(input())
    if num==1:
        login(data_base)
        print("Done")
    elif num==2:
        register(data_base)
    elif num==3:
        exit() 
    else:
        print("Choose either 1 or 2")
        main(data_base)   

In [22]:
main(data_base)

Select one option
1.Already registered
2.New User
3.Exit
1
Fake image
Done


In [36]:
# '''
# def login(data_base):
#     e=0
#     while e<1:
        
#         detector=dlib.get_frontal_face_detector()
#         img_size=96
#         camera=cv2.VideoCapture(0)
#         index=1
#         while(index<=40):
#             success,image=camera.read()
#             d=detector(image,1)
#             for i,d in enumerate(d):
#                 x1=d.top() if d.top() >0 else 0
#                 y1=d.bottom() if d.bottom()>0 else 0
#                 x2=d.left() if d.left()>0 else 0
#                 y2=d.right() if d.right()>0 else 0

#                 face=image[x1:y1,x2:y2]
#                 face=relight(face,random.uniform(0.5,1.5),random.randint(-50,50))
#                 face=cv2.resize(face,(img_size,img_size))
#                 cv2.imwrite('C:\\Users\\Owner\\Desktop\\p2/login_images'+'/'+'li'+str(index)+'.jpg',face)
#                 index=index+1                    
#         camera.release()
#         dir_='C:\\Users\\Owner\\Desktop\\p2/login_images'  
#         img=[]
#         x_dir='C:\\Users\\Owner\\Desktop\\p2\\login_images'
#         #x_dir='C:\\Users\\Owner\\Desktop\p\\face_data\\chandu_2'
#         for i in os.listdir(x_dir):
#             x=np.array(cv2.imread(os.path.join(x_dir,i)))/255
#             x=np.expand_dims(x,axis=0)
#             img.append(x)   
#         pred=[]
#         for i in range(0,40):
#             x=mod.predict(img[i])
#             pred.append(x)
#         p=[]
#         for i in range(0,40):
#             p.append(np.argmax(pred[i]))
#         k=[i for i,j in Counter(p).items() if j==max(Counter(p).values())]

#         if k[0]==1:
#             print("Real image")
#         else:
#             print("Fake image")
#             e=e+1
#             continue
#         k=1
#         pk=[]
#         dat={}
#         for p in os.listdir(dir_):
#             dat[str(p)+'_'+str(k)]=img_to_encoding(os.path.join(dir_,p),model)
#             k=k+1
#         for x,y in [(0,8),(8,16),(16,24),(24,32),(32,40)]:
#             z=np.zeros(128)
#             for j,i in list(dat.items())[x:y]:
#                 z=z+np.abs(i)
#             z=z/8
#         #with open('C:\\Users\\Owner\\Desktop\\py_files/face_data/data_base.pickle', 'rb') as handle:
#          #   data_base = pickle.load(handle)
#         #print('z:',z)
#             pk.append(who_is_it(z,data_base,model))
#         #print ("it's " + str(identity))
#         print('id :',pk)
#         k=[k for k,v in Counter(pk).items() if v==max(Counter(pk).values())]
#         id_=k[0]
#         print("Welcome")
#         page_1(id_)
#         e=e+1
# '''